In [47]:
#prepare to import data
import geopandas as gpd
import pandas as pd
import numpy as np
import fiona
from pathlib import Path
import geopy
import pyogrio
import glob
import os

pd.set_option('display.max_columns', None)
pd.options.display.max_rows = 4000

list_of_files = glob.glob(r'C:\Users\ArianDehnow\norcal_landscape_treatment\JoinedDataExports\*.csv')
fp = max(list_of_files, key=os.path.getctime)

In [48]:
join = pd.read_csv(fp)

In [49]:
join['activity_status'] = join['activity_status'].fillna('N/A')

In [50]:
watersheds = pd.DataFrame(join.groupby(['WatershedGroup','category3','activity_status']).activity_acres.agg('sum'))

In [51]:
watersheds.reset_index(inplace = True)

In [52]:
watersheds = watersheds.pivot_table(index = 'category3',columns = ['WatershedGroup','activity_status'],values = 'activity_acres',dropna = False)

In [53]:
watersheds.reset_index(inplace = True)
watersheds = watersheds[~watersheds['category3'].isin(['Forest and Watershed Health','Other','Restoration - Other'])]
watersheds['category3'] = watersheds.category3.str.replace('Restoration - Wildfire Resilience', 'Restoration')

In [54]:
watersheds.rename(columns={"category3": "Treatment",
                          'activity_status':'Status'},inplace = True )

In [55]:
watersheds.reset_index(inplace = True)

In [56]:
watersheds.drop(columns = ['index'],inplace=True)

C:\Users\ArianDehnow\AppData\Local\Temp\ipykernel_27156\3621944220.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  watersheds.drop(columns = ['index'],inplace=True)


In [57]:
watersheds.fillna(0,inplace = True)

In [58]:
watersheds

WatershedGroup                       Treatment Applegate                   \
activity_status                                   Active    Complete  N/A   
0                Beneficial Fire - Intentional       0.0    0.000000  0.0   
1                            Grazing and Range       0.0    0.000000  0.0   
2                                      Harvest       0.0  387.561236  0.0   
3                             Linear Clearcuts       0.0    0.000000  0.0   
4                   Mechanical Fuels Reduction       0.0    0.000000  0.0   
5                                Reforestation       0.0  220.840014  0.0   
6                                  Restoration       0.0    0.000000  0.0   

WatershedGroup        Feather                             Klamath Watersheds  \
activity_status        Active       Complete          N/A             Active   
0                  888.078531   29281.610329   958.648287        3007.070000   
1                  109.000000    1670.000000     0.000000           0.000000   
2                    0.000000  136664.359151  4259.713801         532.500000   
3                    0.000000      46.860000     0.000000          10.060000   
4                28850.774798   37771.825903  2589.098063       37070.260000   
5                12893.271629  122850.154709    47.840983       14112.546429   
6                    0.000000     508.418359  1948.733555           0.000000   

WatershedGroup                                   McCloud                \
activity_status       Complete           N/A      Active      Complete   
0                 62700.351680  75556.048059    6.000000   9720.530810   
1                  1990.339000      0.000000    0.000000      0.000000   
2                 59900.169597  12893.020177    0.000000  10545.465558   
3                     0.000000      0.000000  427.250000      3.320000   
4                599482.701380  30689.319853  235.450000   6561.809172   
5                 50508.179893   6747.680027  319.510184   6792.742976   
6                   344.427226    665.493183    0.000000   1616.073846   

WatershedGroup                     Pit                              \
activity_status          N/A    Active      Complete           N/A   
0                  91.063317    359.85  39252.305900  17571.049329   
1                   0.000000      0.00      0.000000      0.000000   
2                1777.766549      0.00  20091.885370   8482.037364   
3                   0.000000    233.27      5.030000      0.000000   
4                5821.753961  51870.87  15362.502906   8244.068486   
5                 278.251365   1730.61  18804.752163   1097.645568   
6                  60.544025      0.00   2715.872270    628.588095   

WatershedGroup  Sacramento Headwaters                          Trinity  \
activity_status                Active      Complete        N/A  Active   
0                               63.10   1511.204174   0.000000    1.50   
1                                0.00      0.000000   0.000000    0.00   
2                                0.00  16753.458285   0.000000    0.00   
3                              297.04     18.100000   0.000000   28.18   
4                              654.76   2078.017343  31.917617  237.47   
5                               56.00  14822.002279   0.000000    0.00   
6                                0.00      0.000000   0.000000    0.00   

WatershedGroup                            
activity_status     Complete         N/A  
0                1294.981363   30.066873  
1                   0.000000    0.000000  
2                8627.633417    0.000000  
3                   0.000000    0.000000  
4                1034.619936  165.702804  
5                8587.945995    0.000000  
6                7284.313056    0.000000

In [59]:
for column in watersheds.columns:
    if watersheds[column].dtype == 'float64':
        watersheds[column] = watersheds[column].astype('int')

In [60]:
counties = pd.DataFrame(join.groupby(['county','category3','activity_status']).activity_acres.agg('sum'))

In [61]:
counties.reset_index(inplace = True)

In [62]:
counties = counties.pivot_table(index = 'category3',columns = ['county','activity_status'],values = 'activity_acres',dropna=False)
counties.reset_index(inplace = True)

In [63]:
siskiyou = counties[['category3','Siskiyou']]

In [64]:
siskiyou = siskiyou[~siskiyou['category3'].isin(['Forest and Watershed Health','Other','Restoration - Other'])]
siskiyou['category3'] = siskiyou.category3.str.replace('Restoration - Wildfire Resilience', 'Restoration')
siskiyou.rename(columns={"category3": "Treatment",
                        'Siskiyou':'Siskiyou County'},inplace = True )
siskiyou.reset_index(inplace = True)
siskiyou.drop(columns = ['index'],inplace=True)
siskiyou.fillna(0,inplace = True)
siskiyou['Siskiyou County'] = siskiyou['Siskiyou County'].astype('int')
siskiyou

C:\Users\ArianDehnow\AppData\Local\Temp\ipykernel_27156\3706770569.py:6: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  siskiyou.drop(columns = ['index'],inplace=True)


county                               Treatment Siskiyou County                
activity_status                                         Active Complete    N/A
0                Beneficial Fire - Intentional            3054    73091  63374
1                            Grazing and Range               0     1990      0
2                                      Harvest             532    72200  14907
3                             Linear Clearcuts             734       21      0
4                   Mechanical Fuels Reduction           47648   603359  32753
5                                Reforestation            4527    56777   6698
6                                  Restoration               0     4216    985

In [65]:
overview = siskiyou.merge(watersheds, on = 'Treatment')

C:\Users\ArianDehnow\AppData\Local\Temp\ipykernel_27156\3825167414.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  overview = siskiyou.merge(watersheds, on = 'Treatment')


In [66]:
overview.set_index('Treatment', inplace=True)

In [67]:
overview = overview[[("Siskiyou County", "Complete"),
                    ("Siskiyou County", "Active"),
                    ("Siskiyou County", "N/A"),
                    ("Applegate", "Complete"),
                    ("Applegate", "Active"),
                    ("Applegate", "N/A"),
                    ("Feather", "Complete"),
                    ("Feather", "Active"),
                    ("Feather", "N/A"),
                     ("Klamath Watersheds", "Complete"),
                    ("Klamath Watersheds", "Active"),
                    ("Klamath Watersheds", "N/A"),
                     ("McCloud", "Complete"),
                    ("McCloud", "Active"),
                    ("McCloud", "N/A"),
                     ("Pit", "Complete"),
                    ("Pit", "Active"),
                    ("Pit", "N/A"),
                     ("Sacramento Headwaters", "Complete"),
                    ("Sacramento Headwaters", "Active"),
                    ("Sacramento Headwaters", "N/A"),
                     ("Trinity", "Complete"),
                    ("Trinity", "Active"),
                    ("Trinity", "N/A")
                    ]]

In [68]:
overview

Siskiyou County               Applegate         \
activity_status                      Complete Active    N/A  Complete Active   
Treatment                                                                      
Beneficial Fire - Intentional           73091   3054  63374         0      0   
Grazing and Range                        1990      0      0         0      0   
Harvest                                 72200    532  14907       387      0   
Linear Clearcuts                           21    734      0         0      0   
Mechanical Fuels Reduction             603359  47648  32753         0      0   
Reforestation                           56777   4527   6698       220      0   
Restoration                              4216      0    985         0      0   

                                   Feather              Klamath Watersheds  \
activity_status               N/A Complete Active   N/A           Complete   
Treatment                                                                    
Beneficial Fire - Intentional   0    29281    888   958              62700   
Grazing and Range               0     1670    109     0               1990   
Harvest                         0   136664      0  4259              59900   
Linear Clearcuts                0       46      0     0                  0   
Mechanical Fuels Reduction      0    37771  28850  2589             599482   
Reforestation                   0   122850  12893    47              50508   
Restoration                     0      508      0  1948                344   

                                             McCloud                   Pit  \
activity_status               Active    N/A Complete Active   N/A Complete   
Treatment                                                                    
Beneficial Fire - Intentional   3007  75556     9720      6    91    39252   
Grazing and Range                  0      0        0      0     0        0   
Harvest                          532  12893    10545      0  1777    20091   
Linear Clearcuts                  10      0        3    427     0        5   
Mechanical Fuels Reduction     37070  30689     6561    235  5821    15362   
Reforestation                  14112   6747     6792    319   278    18804   
Restoration                        0    665     1616      0    60     2715   

                                            Sacramento Headwaters             \
activity_status               Active    N/A              Complete Active N/A   
Treatment                                                                      
Beneficial Fire - Intentional    359  17571                  1511     63   0   
Grazing and Range                  0      0                     0      0   0   
Harvest                            0   8482                 16753      0   0   
Linear Clearcuts                 233      0                    18    297   0   
Mechanical Fuels Reduction     51870   8244                  2078    654  31   
Reforestation                   1730   1097                 14822     56   0   
Restoration                        0    628                     0      0   0   

                               Trinity              
activity_status               Complete Active  N/A  
Treatment                                           
Beneficial Fire - Intentional     1294      1   30  
Grazing and Range                    0      0    0  
Harvest                           8627      0    0  
Linear Clearcuts                     0     28    0  
Mechanical Fuels Reduction        1034    237  165  
Reforestation                     8587      0    0  
Restoration                       7284      0    0

In [69]:
import datetime
name = f'overview_{datetime.datetime.now().strftime("%H%M_%m%d%Y")}.csv'
export_path = Path(r"C:\Users\ArianDehnow\norcal_landscape_treatment\OverviewExports")

overview.to_csv(export_path / name)